In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
a=pd.read_csv("WineQT.csv")
df=pd.DataFrame(a)
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4


In [4]:
df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
Id                      0
dtype: int64

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
df.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality', 'Id'],
      dtype='object')

In [8]:
X=df.drop(['quality','Id'],axis='columns')
y=df['quality']

In [9]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

In [10]:
X_train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
41,7.3,0.39,0.31,2.4,0.074,9.0,46.0,0.99620,3.41,0.54,9.4
156,7.8,0.34,0.37,2.0,0.082,24.0,58.0,0.99640,3.34,0.59,9.4
1123,6.4,0.38,0.14,2.2,0.038,15.0,25.0,0.99514,3.44,0.65,11.1
535,8.3,0.65,0.10,2.9,0.089,17.0,40.0,0.99803,3.29,0.55,9.5
549,8.8,0.59,0.18,2.9,0.089,12.0,74.0,0.99738,3.14,0.54,9.4
...,...,...,...,...,...,...,...,...,...,...,...
752,6.6,0.52,0.08,2.4,0.070,13.0,26.0,0.99358,3.40,0.72,12.5
814,7.8,0.39,0.42,2.0,0.086,9.0,21.0,0.99526,3.39,0.66,11.6
702,7.5,0.43,0.30,2.2,0.062,6.0,12.0,0.99495,3.44,0.72,11.5
450,7.9,0.35,0.21,1.9,0.073,46.0,102.0,0.99640,3.27,0.58,9.5


In [11]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [21]:
import mlflow
from mlflow.models.signature import infer_signature
import dagshub
import logging
import sys

In [15]:
import mlflow

In [18]:
model=ElasticNet()


In [22]:
logging.basicConfig(level=logging.WARN)
logger=logging.getLogger(__name__)

In [24]:
alpha = 0.5
l1_ratio = 0.5

In [28]:
model.fit(X_train,y_train)

ElasticNet(alpha=0.5)

In [29]:
y_pred=model.predict(X_test)

In [49]:
def eval_metrics(y_test,y_pred):
    rmse=np.sqrt(mean_squared_error(y_test,y_pred))
    mae=mean_absolute_error(y_test,y_pred)
    r2s=r2_score(y_test,y_pred)
    #print(f"RMSE: {rmse}, MAE: {mae},score:{r2s}")
    return rmse,mae,r2s


In [44]:
eval_metrics(y_test,y_pred)

In [46]:
from urllib.parse import urlparse

In [55]:
import dagshub
dagshub.init(repo_owner='vasanthk77', repo_name='MLOPS_FOUNDATION', mlflow=True)

Initialized MLflow to track repo "vasanthk77/MLOPS_FOUNDATION"

Repository vasanthk77/MLOPS_FOUNDATION initialized!

In [56]:

import dagshub
dagshub.init(repo_owner='vasanthk77', repo_name='MLOPS_FOUNDATION', mlflow=True)
with mlflow.start_run():
    model=ElasticNet(alpha=alpha,l1_ratio=l1_ratio)
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    (rmse,mae,score)=eval_metrics(y_test,y_pred)

    mlflow.log_param("alpha", alpha)  # Use log_param (singular) for a single key-value pair
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)  # Correct for metrics
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2", score)


    ## remore server 
    remote_server_url="https://github.com/vasanthk77/MLOPS_FOUNDATION.git"
    mlflow.set_tracking_uri(remote_server_url)

    tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme


    if tracking_url_type_store != "file":
        mlflow.sklearn.log_model(model,"model",registered_model_name="ElasticNetModle")
    else:
        mlflow.sklearn.log_model(model,"model")











    

Initialized MLflow to track repo "vasanthk77/MLOPS_FOUNDATION"

Repository vasanthk77/MLOPS_FOUNDATION initialized!

MlflowException: API request to endpoint /api/2.0/mlflow/runs/create failed with error code 422 != 200. Response body: '<!DOCTYPE html>
<html>
  <head>
    <meta http-equiv="Content-type" content="text/html; charset=utf-8">
    <meta http-equiv="Content-Security-Policy" content="default-src 'none'; base-uri 'self'; connect-src 'self'; form-action 'self'; img-src 'self' data:; script-src 'self'; style-src 'unsafe-inline'">
    <meta content="origin" name="referrer">
    <title>Oh no &middot; GitHub</title>
    <meta name="viewport" content="width=device-width">
    <style type="text/css" media="screen">
      body {
        background-color: #f6f8fa;
        color: #24292e;
        font-family: -apple-system,BlinkMacSystemFont,Segoe UI,Helvetica,Arial,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol;
        font-size: 14px;
        line-height: 1.5;
        margin: 0;
      }

      .container { margin: 50px auto; max-width: 600px; text-align: center; padding: 0 24px; }

   